# Big Data Modeling and Management Assigment - Homework 1

## 🍺 The Beer project  🍺 

As it was shown in classes, graph databases are a natural way of navegating distinct types of data. For this first project we will be taking a graph database to analyse beer and breweries!   

_For reference the dataset used for this project has been extracted from [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. Even though the author does not present metada on the origin of the data it is probably a collection of open data from places like [beeradvocate](https://www.beeradvocate.com/)_.



### Table of Contents

* [Dataset description](#Problemdescription)
* [All The Functions you'll need to run queries in Neo4J](#AllTheFunctionsyoullneedtorunqueriesinNeo4J)
* [Understanding the Database](#UnderstandingtheDatabase)
* [1. How many different countries exist in the database?](#q1)
* [2. Most reviews](#q2)
* [3. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?](#q3)
* [4. Which Portuguese brewery has the most beers?](#q4)
* [5. From those beers (the ones produced in the brewery from the previous question), which has the most reviews?](#q5)
* [6. On average how many different beer styles does each brewery produce?](#q6)
* [7. Which brewery produces the strongest beers according to ABV?](#q7)
* [8. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?](#q8)
* [9. Using Graph Algorithms answer one of the following questions](#q9)
* [10. Using Graph Algorithms answer one of the following questions](#q10)
* [11. If you had to pick 3 beers to recommend using only this database, which would you pick and why?](#q11)

### Problem description <a class="anchor" id="Problemdescription"></a>

Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions. Submit the results by following the instructions


### Questions

1. How many different countries exist in the database?
1. Most reviews:  
    1. Which `Beer` has the most reviews?  
    1. Which `Brewery` has the most reviews for its beers? [Hint: 5-node path]
    1. Which `Country` has the most reviews for its beers? [Hint: 5-node path]
1. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?
1. Which Portuguese brewery has the most beers?
1. From those beers (the ones returned from the previous question), which has the most reviews?
1. On average how many different beer styles does each brewery produce?
1. Which brewery produces the strongest beers according to ABV? [Hint: database has NaN values]
1. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try? (Justify your answer well!) [Hint: database has NaN values]
1. Using Graph Algorithms answer **one** of the following questions: [Hint: make sure to clear the graph before using it again]
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    2. Which beer is the most influential when considering beers are conected by users who review them? [Please use limit of 1000 on beer-review-user path]]
    3. Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users has the biggest community? [Please use limit of 1000 on beer-review-user path]]
1. Using Graph Algorithms answer **one** of the following questions:
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`? [Hint:inspect two subsets: with and without the beer in question]
    2. Which user is the most influential when it comes to reviews made?
1. If you had to pick 3 beers to recommend using only this database, which would you pick and why? (Justify your answer well!) [Hint: database has NaN values]


Questions 8 to 10 are somewhat open, which means we'll also be evaluating the reasoning behind your answer. So there aren't necessarily bad results there are only wrong criteria, explanations or execution. 
 
### Groups  

Groups should have 4 to 5 people.
You should register your group on **moodle**.

### Submission      

The code used to produce the results and respective explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of March 13**

### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and simplicity of the solution.  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

## Loading the Database

#### Be sure that you **don't have** the neo4j docker container from the classes running (you can Stop it in the desktop app or with the command "`docker stop Neo4JLab`")


The default container does not have any data whatsoever, we will have to load a database into our docker image:
- Download and unzip the `Neo4JHWData` file provided in Moodle.
- Copy the path of the `Neo4JHWData` folder of the unziped file, e.g. `C:\Users\###\Big Data Modelling and Management\Nuno\HM1\Neo4JHWData`.
- Download and unzip the `Neo4JPlugins` file provided in Moodle.
- Copy the path of the `Neo4JPlugins` folder of the unziped file, e.g. `C:\Users\####\Big Data Modelling and Management\Nuno\Neo4JPlugins`.
- Change the code bellow accordingly. As you might have noticed, you do not have a user called `nunoa`, please use the appropriate path that you got from the previous step. Be sure that you have a neo4j docker container running: \

`docker run --name Neo4JHW -p 7474:7474 -p 7687:7687 -d -v "C:\Big Data Modelling and Management\Neo4JPlugins":/plugins -v "C:\Big Data Modelling and Management\Neo4JHWData\Neo4JHWData\data":/data --env NEO4J_AUTH=neo4j/test --env NEO4J_dbms_connector_https_advertised__address="localhost:7473" --env NEO4J_dbms_connector_http_advertised__address="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised__address="localhost:7687" --env NEO4J_dbms_security_procedures_unrestricted=gds.* --env NEO4J_dbms_security_procedures_allowlist=gds.* neo4j:4.4.5`

- Since Neo4j is trying to recognize a new database folder, this might take a bit (let's say 3 minutes), so don't worry.

In [3]:
#importing all needed packages

from neo4j import GraphDatabase
from pprint import pprint

In [4]:
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="test"

In [5]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), )

### All The Functions you'll need to run queries in Neo4J <a class="anchor" id="AllTheFunctionsyoullneedtorunqueriesinNeo4J"></a>

In [6]:
def execute_read(driver, query):    
    with driver.session(database="neo4j") as session:
        result = session.read_transaction(lambda tx, query: list(tx.run(query)), query)
    return result

### Understanding the Database <a class="anchor" id="UnderstandingtheDatabase"></a>

In [12]:
query = """
        call db.labels();
    """

result = execute_read(driver, query)

pprint(result)

[<Record label='COUNTRIES'>,
 <Record label='CITIES'>,
 <Record label='BREWERIES'>,
 <Record label='BEERS'>,
 <Record label='REVIEWS'>,
 <Record label='STYLE'>,
 <Record label='USER'>]


In [13]:
query = """
        CALL db.relationshipTypes();
    """

result = execute_read(driver, query)

pprint(result)

[<Record relationshipType='REVIEWED'>,
 <Record relationshipType='BREWED'>,
 <Record relationshipType='IN'>,
 <Record relationshipType='HAS_STYLE'>,
 <Record relationshipType='POSTED'>]


# Submission

GROUP NUMBER:

**Group 20**

GROUP MEMBERS:

|STUDENT NAME|
|---|
|Daniel Torres|
|Yousef Ebrahimi|
|Evans Onorieru|
|João Pólvora|
|Luca Loureiro|

#### 1. How many different countries exist in the database? <a class="anchor" id="q1"></a>

In [17]:
#We used the count function with the DISTINCT keyword, 
#the query only counts unique instances of the name 
#among the nodes that match.

query = """
    MATCH (c:COUNTRIES)
    RETURN count(DISTINCT c.name) AS Countries_In_DB
"""

result = execute_read(driver, query)
pprint(result)


[<Record Countries_In_DB=200>]


#### 2. Most reviews:   <a class="anchor" id="q2"></a>
    A) Which `Beer` has the most reviews?  
    B) Which `Brewery` has the most reviews for its beers?
    C) Which `Country` has the most reviews for its beers? 

In [18]:
#A) 

#in order to find the most review 'Beer' we used a MATCH clause
#to match BEERS nodes that are connected to REVIEWS nodes by 
#the REVIEWED relationship.

query = """
        MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)
        RETURN b.name AS beer, count(r) AS number_of_reviews
        ORDER BY number_of_reviews DESC
        LIMIT 1

"""

result = execute_read(driver, query)
pprint(result)

[<Record beer='IPA' number_of_reviews=31387>]


In [19]:
#B) 

#Following the path BREWERIES-brewed-BEERS-reviews-REVIEWS
#count number of reviews grouped by brewery.name

query = """
        MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS),(b:BEERS)-[:REVIEWED]->(r:REVIEWS)
        RETURN br.name, count(r) as num_reviews
        ORDER BY num_reviews DESC
        LIMIT 1
"""

result = execute_read(driver, query)
pprint(result)


[<Record br.name='Sierra Nevada Brewing Co.' num_reviews=175161>]


In [20]:
#C) 

#Following the nodes REVIEWS-review-BEERS-brewed-BREWERIES-in-CITIES-in-COUNTRIES
#count number of reviews by country.name

query = """

        MATCH (co:COUNTRIES)<-[:IN]-(c:CITIES),(c:CITIES)<-[:IN]-(br:BREWERIES),
        (br:BREWERIES)-[:BREWED]->(b:BEERS),(b:BEERS)-[:REVIEWED]->(r:REVIEWS)
        RETURN co.name, count(r) as num_reviews
        ORDER BY num_reviews DESC
        LIMIT 1

"""

result = execute_read(driver, query)
pprint(result)


[<Record co.name='US' num_reviews=7675804>]


#### 3. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?   <a class="anchor" id="q3"></a>

In [21]:
#finding the CTJman user who shared reviews at most for the same beers by matching the reviews to beers and
#then count them. Actually we checked the relationship between two users who reviews the beers.

query = """
        MATCH (u:USER)<-[:POSTED]-(r:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:REVIEWED]->(re:REVIEWS)-[:POSTED]->(user:USER)
        where u.name= "CTJman"
        RETURN user.name AS User, COUNT(DISTINCT b) AS most_shared_reviews
        order by most_shared_reviews desc
        LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)


[<Record User='acurtis' most_shared_reviews=1428>]


#### 4. Which Portuguese brewery has the most beers?   <a class="anchor" id="q4"></a>

In [22]:
#Following the nodes BEERS-brewed-BREWERIES-in-CITIES-in-COUNTRIES
#and then we use size funtcion to Count the paths matching the Brewed with Beers
#filter country.name = 'PT'

query = """
        MATCH (b:BREWERIES)-[:IN]->(:CITIES)-[:IN]->(:COUNTRIES {name: "PT"})
        WITH b, size((b)-[:BREWED]->(:BEERS)) AS num_beers
        ORDER BY num_beers DESC
        LIMIT 1
        RETURN b.name AS brewery_name, num_beers AS num_beers

    """

result = execute_read(driver, query)

pprint(result)

[<Record brewery_name='Dois Corvos Cervejeira' num_beers=40>]


#### 5. From those beers (the ones produced in the brewery from the previous question), which has the most reviews?   <a class="anchor" id="q5"></a>

In [23]:
#Following the nodes BREWERIES-brew-BEERS-reviewed-REVIEWS
#count the number of reviews nodes
#associated with breweries.name ='Dois Corvos'

query = """
    MATCH (br:BREWERIES{name:'Dois Corvos Cervejeira'})-[:BREWED]->(b:BEERS),(b:BEERS)-[:REVIEWED]->(r:REVIEWS)
    RETURN b.name, count(r) as num_reviews
    ORDER BY num_reviews DESC
    LIMIT 1
"""

result = execute_read(driver, query)
pprint(result)

[<Record b.name='Finisterra' num_reviews=10>]


#### 6. On average how many different beer styles does each brewery produce?   <a class="anchor" id="q6"></a>

In [45]:
#Following the path BREWERIES-brewed-BEERS-has_style-STYLE
#count the number of different styles each brewery produce and 
#compute the average on the count
#Use distinct because a brewery can produce the same style on different beers

query = """
        MATCH (br:BREWERIES)-[:BREWED]->(:BEERS)-[:HAS_STYLE]->(s:STYLE)
        WITH br.name AS Brewery, COUNT(DISTINCT s) AS count
        RETURN avg(count) AS Avg_Beer_in_Brewery
    
"""

result = execute_read(driver, query)

pprint(result)


[<Record Avg_Beer_in_Brewery=10.66997731592179>]


#### 7. Which brewery produces the strongest beers according to ABV?   <a class="anchor" id="q7"></a>

In [25]:
#Following the path BREWERIES-brew-BEERS
#calculate abv average, excluding nan
#order by average result
#b.abv <> 'nan' means consider abv different from nan

query = """
    MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)
    WHERE b.abv <> 'nan'
    RETURN br.name, avg(toFloat(b.abv)) as avg_score
    ORDER BY avg_score DESC
    LIMIT 1
"""

result = execute_read(driver, query)

pprint(result)


[<Record br.name='1648 Brewing Company Ltd' avg_score=25.57777777777778>]


#### 8. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?   <a class="anchor" id="q8"></a>

In [26]:
#Following the path BEERS-has_style-STYLE and BEERS-reviewed-REVIEWS
#sum look and smell of each review by style
#order the sum to get the one with high score

query = """
        MATCH (s:STYLE)<-[:HAS_STYLE]-(b:BEERS),(b:BEERS)-[:REVIEWED]->(r:REVIEWS)
        WHERE s.name <> 'nan' AND r.smell <> 'nan' AND r.look <> 'nan'
        RETURN s.name AS Style, round((avg(toFloat(r.smell))) + (avg(toFloat(r.look))),1) AS Aroma&Appearance_Score, count(r) AS Review
        ORDER BY Aroma&Appearance_Score DESC
        LIMIT 1
"""
result = execute_read(driver, query)
pprint(result)


[<Record Style='New England IPA' Aroma_Appearance_Score=8.8 Review_Count=110696>]


#### 9. Using Graph Algorithms answer **one** of the following questions:   <a class="anchor" id="q9"></a>
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    2. Which beer is the most influential when considering beers are conected by users who review them?
    3. Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users has the biggest community?

In [22]:
# Step 0 - Clear graph, graph names need to be unique

try:
    query = """
            CALL gds.graph.drop('BEER') YIELD graphName;
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

C:\Users\lucal\AppData\Local\Temp\ipykernel_14896\1008916601.py:3: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(lambda tx, query: list(tx.run(query)), query)


ClientError('Failed to invoke procedure `gds.graph.drop`: Caused by: java.util.NoSuchElementException: Graph with name `BEER` does not exist on database `neo4j`. It might exist on another database.')


In [23]:
# Step 1 - Create an appropriate graph

try:
    query = """
        CALL gds.graph.project.cypher(
            'beer_influential',
            'MATCH (b:BEERS) RETURN id(b) AS id',
            'MATCH (u:User)<-[:POSTED]-(r:REVIEWS)<-[:REVIEWED]-(b:BEERS) RETURN id(u) AS source, r.id as weight, id(b) AS target'
            )
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

C:\Users\lucal\AppData\Local\Temp\ipykernel_14896\1008916601.py:3: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(lambda tx, query: list(tx.run(query)), query)


[<Record nodeQuery='MATCH (b:BEERS) RETURN id(b) AS id' relationshipQuery='MATCH (u:User)<-[:POSTED]-(r:REVIEWS)<-[:REVIEWED]-(b:BEERS) RETURN id(u) AS source, r.id as weight, id(b) AS target' graphName='beer_influential' nodeCount=358873 relationshipCount=0 projectMillis=2759>]


In [25]:
# Step 2 - Run the algorithm

try:
    query = """
        CALL gds.pageRank.stream('beer_influential', {relationshipWeightProperty:'weight'})
            YIELD nodeId, score
            RETURN gds.util.asNode(nodeId).name AS name, score
            ORDER BY score desc
            limit 5
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

C:\Users\lucal\AppData\Local\Temp\ipykernel_14896\1008916601.py:3: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(lambda tx, query: list(tx.run(query)), query)


[<Record name='Konrads Stout Russian Imperial Stout' score=0.15000000000000002>,
 <Record name='Scottish Right' score=0.15000000000000002>,
 <Record name='MegaMeow Imperial Stout' score=0.15000000000000002>,
 <Record name='Peaches-N-Cream' score=0.15000000000000002>,
 <Record name='Olde Cogitator' score=0.15000000000000002>]


#### 10. Using Graph Algorithms answer **one** of the following questions:   <a class="anchor" id="q10"></a>
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`?
    2. Which user is the most influential when it comes to reviews made?

In [11]:
#step 0

try:
    query = """
            CALL gds.graph.drop('Beer_Graph') YIELD graphName;
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

C:\Users\lucal\AppData\Local\Temp\ipykernel_14896\1008916601.py:3: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(lambda tx, query: list(tx.run(query)), query)


ClientError('Failed to invoke procedure `gds.graph.drop`: Caused by: java.util.NoSuchElementException: Graph with name `Beer_Graph` does not exist on database `neo4j`. It might exist on another database.')


In [12]:
#Step 1 - Create an appropriate graph

try:
    query = """
        CALL gds.graph.project('Beer_Graph',['USER', 'REVIEWS'],{POSTED:{}})
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

C:\Users\lucal\AppData\Local\Temp\ipykernel_14896\1008916601.py:3: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(lambda tx, query: list(tx.run(query)), query)


[<Record nodeProjection={'REVIEWS': {'label': 'REVIEWS', 'properties': {}}, 'USER': {'label': 'USER', 'properties': {}}} relationshipProjection={'POSTED': {'orientation': 'NATURAL', 'aggregation': 'DEFAULT', 'type': 'POSTED', 'properties': {}}} graphName='Beer_Graph' nodeCount=9238063 relationshipCount=9073109 projectMillis=23188>]


In [20]:
#Step 2
#As we can see the most influencial user is the person who posted
#a lot of reviews
#and we used the page rank code from lab 3 to find
#https://neo4j.com/docs/graph-data-science/current/algorithms/page-rank/ 

try:
    query = """
        CALL gds.pageRank.stream('Beer_Graph')
        YIELD nodeId, score
        RETURN gds.util.asNode(nodeId).name AS Username, round(score, 2) AS most_influential
        ORDER BY most_influential DESC 
        LIMIT 1
        """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

C:\Users\lucal\AppData\Local\Temp\ipykernel_14896\1008916601.py:3: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(lambda tx, query: list(tx.run(query)), query)


[<Record Username='Sammy' most_influential=1759.27>]


#### 11. If you had to pick 3 beers to recommend using only this database, which would you pick and why?   <a class="anchor" id="q11"></a>

In [39]:
#We checked the beers with score over 4 becuase the higher score means the better ones.
#Also based on this ranking: https://www.beeradvocate.com/beer/top-rated/
#top best ranking beers are from USA so, we choose us as the best country for finding the 3 best beers

query = """
     MATCH (c:COUNTRIES)<-[]-(:CITIES)<-[]-(:BREWERIES)-[]->(b:BEERS)-[r:REVIEWED]->(re:REVIEWS)
     WHERE re.score <> 'nan' AND c.name ='US'
     WITH b.name as Beer_Name, round(avg(toFloat(re.score)),1) as Avg_Score, count(re) AS Best_Review
     WHERE Avg_Score > 4
     Return Beer_Name, Avg_Score, Best_Review
     ORDER BY Best_Review DESC
     LIMIT 3
"""

result = execute_read(driver, query)
pprint(result)


[<Record Beer_Name='Breakfast Stout' Avg_Score=4.5 Best_Review=17573>,
 <Record Beer_Name='90 Minute IPA' Avg_Score=4.3 Best_Review=15947>,
 <Record Beer_Name='Pliny The Elder' Avg_Score=4.6 Best_Review=14927>]
